# Разреженные признаки

Нам понадобятся пакеты nltk и spacy (на самом деле нет). Можете уже начать их устанавливать.

In [1]:
import re
import spacy
import scipy
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Примеры источников разреженных признаков:
* категориальные признаки 
* текст

## Категориальные признаки

Рассмотрим таблицу [```winemag.csv```](https://yadi.sk/d/bi2cwBBO_y6cCA), которая содержит описания вин:

In [2]:
data = pd.read_csv('winemag.csv', index_col=0, na_filter=False)
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,,Sicily & Sardinia,Etna,,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,,,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,,Alexander Peartree,,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


В таблице 13 столбцов с признаками. Какие из них являются категориальными? 

Во-первых, обычно это столбцы содержащие текстовые значения. Исключением могут быть, например, некоторые идентификаторы (например, цифровой код региона). Следовательно, в качестве кандидатов остаются: ```country```, ```description```, ```designation```, ```province```, ```region_1```, ```region_2```, ```taster_name```, ```taster_twitter_handle```, ```title```, ```variety``` и ```winery```.

Во-вторых, столбцы с небольшим числом уникальных значений:

In [3]:
data.nunique()

country                      44
description              119955
designation               37980
points                       21
price                       391
province                    426
region_1                   1230
region_2                     18
taster_name                  20
taster_twitter_handle        16
title                    118840
variety                     708
winery                    16757
dtype: int64

Итак, например страна-производитель является категориальным признаком.

Один из самых простых способов кодирования является **one-hot-кодирование**: для кодируемого категориального признака создаются $N$ новых признаков, где $N$ - число категорий. Каждый $i$-й новый признак - бинарный характеристический признак $i$-й категории.

Воспользуемся [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) (превращает категории в числа) и [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) (превращает числа в векторы) для преобразования названий стран в one-hot-вектор:

In [4]:
countries = data.country
countries = LabelEncoder().fit_transform(countries)
countries = OneHotEncoder(categories='auto').fit_transform(countries[:, np.newaxis])

type(countries)

scipy.sparse.csr.csr_matrix

## Разреженные матрицы

После кодирования мы получили разреженную матрицу признаков. Существует много типов разреженных матриц, каждый из которых предоставляет разные гарантии на операции.

* ```scipy.sparse.coo_matrix```
* ```scipy.sparse.csc_matrix```
* ```scipy.sparse.csr_matrix```
* ```scipy.sparse.bsr_matrix```
* ```scipy.sparse.lil_matrix```
* ```scipy.sparse.dia_matrix```
* ```scipy.sparse.dok_matrix```

Подробнее про [устройство разреженых матрицы](http://www.netlib.org/utk/people/JackDongarra/etemplates/node372.html)

### scipy.sparse.coo_matrix

* Используется как хранилище данных
* Поддерживает быструю конвертацию в любой формат
* Не поддерживает индексацию
* Поддерживает ограниченый набор арифметических операций

### scipy.sparse.csc_matrix

* Хранит данные поколоночно
* Быстрое получение значений отдельных колонок

### scipy.sparse.csr_matrix

* Хранит данные построчно
* Быстрое получение значений отдельных строк

### scipy.sparse.bsr_matrix

* Подходит для разреженных матриц с плотными подматрицами

### scipy.sparse.lil_matrix

* Подходит для создания разреженных матриц поэлементно
* Для последующих матричных операций лучше сконвертировать в ```csr_matrix``` или ```csc_matrix```

Библиотека ```scipy.sparse``` содержит методы, позволяющие работать с разреженными матрицами. Подробнее про операции с разрежеными матрицами на сайте [scipy](https://docs.scipy.org/doc/scipy/reference/sparse.html).

## Предсказание оценки вин

В качестве категориальных признаков возьмём: country, province и variety

Попробуем предсказать оценку выставленную винам. Оценки в таблице варьируются от 80 до 100.

Решим задачу трехклассовой классификации качества вин. Установим значения целевой переменной в LOW, MID или HIGH в зависимости от оценки:

In [5]:
data['quality'] = 'MID'
data.loc[data['points'] <= 85, 'quality'] = 'LOW'
data.loc[data['points'] > 90, 'quality'] = 'HIGH'

In [6]:
data['quality'].value_counts()

MID     74376
HIGH    33635
LOW     21960
Name: quality, dtype: int64

In [7]:
data.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery', 'quality'],
      dtype='object')

И, собственно, попробуем перебрать все подмножества категориальных признаков и выбрать лучшее:

In [8]:
Y = data['quality'].values
countries = OneHotEncoder(categories='auto').fit_transform(LabelEncoder()\
                                            .fit_transform(data.country)[:, np.newaxis])[:, 1:]
provinces = OneHotEncoder(categories='auto').fit_transform(LabelEncoder()\
                                            .fit_transform(data.province)[:, np.newaxis])[:, 1:]
varieties = OneHotEncoder(categories='auto').fit_transform(LabelEncoder()\
                                            .fit_transform(data.variety)[:, np.newaxis])[:, 1:]
features = [('country', countries), ('province', provinces), ('variety', varieties)]

names = []
accuracy_scores = []
for subset_features in tqdm(list(itertools.chain(*[itertools.combinations(features, n) 
                                                   for n in range(1, 4)]))):
    subset_names, subset_features = zip(*subset_features)
    names.append('; '.join(subset_names))
    
    X = scipy.sparse.hstack(subset_features)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
    
    lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000).fit(X_train, Y_train)
    Y_pred = lr.predict(X_test)

    accuracy_scores.append(accuracy_score(Y_pred, Y_test))

pd.DataFrame({'Accuracy':accuracy_scores}, index=names).sort_values('Accuracy')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


,Accuracy
country,0.572148
variety,0.574223
country; variety,0.576858
province,0.577790
country; province,0.578187
province; variety,0.579212
country; province; variety,0.584412


## Извлечение признаков из текстов

Но у нас ещё остались неучтёнными отзывы сомелье! Непорядок.

* токенизируем
* нормализируем ([стемминг и/или лемматизация](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html))

Для работы лемматизации английского текста можно воспользоваться библиотекой [SpaCy]( https://spacy.io/). Также, для работы с текстом удобно использовать библиотеку nltk:

In [9]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag_sents, pos_tag
from nltk.corpus import wordnet as wn

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']


def penn_to_wn(tag):
    if is_adjective(tag):
        return wn.ADJ
    elif is_noun(tag):
        return wn.NOUN
    elif is_adverb(tag):
        return wn.ADV
    elif is_verb(tag):
        return wn.VERB
    return wn.NOUN

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Jeniamakarchik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Jeniamakarchik/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Jeniamakarchik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
description_lemma = []
lemmer = WordNetLemmatizer()
for sent in tqdm(data.description):
    words = []
    for token, pos in pos_tag(nltk.word_tokenize(sent), lang='eng'):
        wn_pos = penn_to_wn(pos)
        words.append(lemmer.lemmatize(token, wn_pos))
    description_lemma.append(' '.join(words))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Hint: вам может пригодиться строчка ```sudo python3 -m spacy download en```

In [11]:
nlp = spacy.load('en')
# По-хорошему надо так
description_lemma = [' '.join([token.lemma_ for token in nlp(text)]) 
                     for text in tqdm(data.description)]
# description_lemma = [' '.join([token for token in text.split(' ')]) 
#                      for text in tqdm(data.description)]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [12]:
description_lemma[0]

'aroma include tropical fruit , broom , brimstone and dry herb . the palate be not overly expressive , offer unripened apple , citrus and dry sage alongside brisk acidity .'

In [13]:
len(description_lemma)

129971

Ну пусть мы каждое слово привели к нормальной форме. Но как описать текст? Какие фичи посчитать?

### Bag of Words

Cоздаем вектор длиной в словарь, для каждого слова считаем количество вхождений в текст и подставляем это число на соответствующую позицию в векторе.

Построим модель BOW с помощью [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html):

In [14]:
vectorizer = CountVectorizer().fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: ', len(vocabulary))

description_count = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: ', ' '.join(top_tokens))

Размер словаря:  26922
Top-10 слов:  and the be of with pron this wine flavor fruit


Видно, что большая часть из топ-10 слов является не информативными - стоп-словами. Что бы они не участвовали в представление, в конструктор CountVectorizer в качестве параметра можно передать список стоп-слов:

In [15]:
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Jeniamakarchik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
stop_words = set(stopwords.words('english'))
vectorizer = CountVectorizer(stop_words=stop_words).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: ', len(vocabulary))

description_count = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: ', ' '.join(top_tokens))

Размер словаря:  26825
Top-10 слов:  pron wine flavor fruit finish aroma palate acidity cherry tannin


Чтобы сжать векторное представление, можно "отбросить" редкие слова:

In [17]:
vectorizer = CountVectorizer(stop_words=stop_words, min_df=3).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
description_count

Размер словаря: 13585


<129971x13585 sparse matrix of type '<class 'numpy.int64'>'
	with 3253123 stored elements in Compressed Sparse Row format>

### Tf-Idf

Cлова, которые редко встречаются в корпусе (во всех рассматриваемых документах этого набора данных), но присутствуют в этом конкретном документе, могут оказаться более важными. Тогда имеет смысл повысить вес более узкотематическим словам, чтобы отделить их от общетематических. Этот подход называется [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf).

Значение Tf-Idf для каждого пары документ-слово состоит из двух компонент:
* Term frequency — встречаемость слова в документе

$$tf(t, d) = \frac{n_{t, d}}{\sum_{k\in d} n_{k,d}}$$

* Inverse Document frequency — логарифм обратной доли документов в которых встретилось данное слово

$$idf(t, D) = \log \frac{ \mid D \mid}{\mid \{ d_i \in D \mid t \in d_i \} \mid}$$

* Tf-Idf — кобминация tf и idf

$$ TfIdf(t, d, D) = tf(t, d) * idf(t, D)$$

**Задание 1 (0.5 балла)**. Воспользуйтесь [TfidfVectorizer'ом](https://scikit-learn.org/0.20/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) и посчитайте фичи. Выведите топ-10 слов по убыванию TF-IDF.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=3)
X_tfidf = tfidf_vectorizer.fit_transform(description_lemma)
terms = tfidf_vectorizer.get_feature_names()
cum_sums = X_tfidf.sum(axis=0) # sum frequences through docs

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append((term, cum_sums[0,col]))

ranking = pd.DataFrame(data, columns=['term','rank'])
print(ranking.sort_values('rank', ascending=False)[:10])

          term         rank
9384      pron  7097.984600
13368     wine  5827.117233
4759    flavor  4932.304836
5057     fruit  4897.980632
376    acidity  3560.532001
864      aroma  3515.070123
4685    finish  3507.807587
8604    palate  3359.468855
3849     drink  3354.613732
2390    cherry  3289.798799


Но вернёмся в вину!

Добавляем к категориальным признакам, признаки извлечённые из описаний сомелье:

In [20]:
X = scipy.sparse.hstack([countries, provinces, varieties, description_count])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

lr = LogisticRegression().fit(X_train, Y_train)
Y_pred = lr.predict(X_test)

print ('Accuracy: ', accuracy_score(Y_pred, Y_test))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy:  0.7648924016693479


**Задание 2 (0.5 балла)**. Воспользуйтесь посчитанными TF-IDF фичами и проверьте, какое качество предсказания будет, если использовать только их.

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, Y, test_size=0.33)

lr = LogisticRegression().fit(X_train, Y_train)
Y_pred = lr.predict(X_test)

print ('Accuracy: ', accuracy_score(Y_pred, Y_test))

Accuracy:  0.7402252220745611


## Vowpal Wabbit

Довольно часто бывает, что данных очень много и обучаться приходится на выборках, которые не помещаются в память. А также, довольно часто хорошее качество можно получить благодаря простым линейным моделям, при условии, что были хорошо отобраны и сгенерированы признаки.

Важным достоинством линейных методов является то, что при обучении можно добиться того, что настройка параметров алгоритма (т.е. этап обновления весов) будет производится каждый раз при добавлении нового обьекта. Данные методы машинного обучения в литературе часто также называют Online Machine Learning. При этом не нужно хранить все обьекты одновременно в памяти теперь просто не нужно.

На сегодняшний день одной из самых известных реализаций таких методов является пакет [Vowpal Wabbit](https://github.com/JohnLangford/vowpal_wabbit):
![](https://cdn.dribbble.com/users/261617/screenshots/3146111/vw-dribbble.png)

* Можно обучать только линейные модели. Увеличивать качество методов, можно за счет добавления новых признаков и подгонки функции потерь

* Обучающая выборка обрабатывается с помощью стахостического оптимизатора, благодаря чему можно обучаться на выборках, которые не помещаются в память

* Можно обрабатывать большое количество признаков за счет их хэширования (так называемый hashing trick), бладаря чему можно обучать модели даже в случаях, когда полный набор весов просто не помещается в памяти

* Поддерживается режим активного обучения, при котором обьекты обучающей выборки можно подавать даже с нескольких машин по сети

* Обучение может быть распараллелено на несколько машин

### Установка

* ```apt-get install vowpal-wabbit```

### Формат данных

Label [weight] |Namespace Feature ... |Namespace ...

* ```Label``` - метка класса для задачи классификации или действительное число для задачи регрессии
* ```weight``` - вес объекта, по умолчанию у всех 1
* ```Namespace``` - все признаки разбиты на области видимости, может использоваться для раздельного использования или создания квадратичных признаков между областями
* ```Feature``` - ```string[:value]``` или ```int[:value]``` строки будут хешированы, числа будут использоваться как индекс в векторе признаков. ```value``` по умолчанию равно $1$

### Параметры

**Hashing trick**

Вводится функция $h$, с помощью которой получается индекс для записи значения в вектор признаков объекта.

$$h : F \rightarrow \{0, \dots, 2^b - 1\}$$

С помощью ```--b``` можно задавать размер области значений хеш-функции. Чем больше значение ```b```, тем меньше вероятность получить коллизии при хешировании признаков.


**Оптимизация**

Может использовать ```SGD``` или ```L-BFGS``` (квази-ньютоновский метод второго порядка, подробнее про работу [оптимизации](http://aria42.com/blog/2014/12/understanding-lbfgs))

По умолчанию используется ```SGD```. ```L-BFGS``` включается с помощью ```--bfgs```, работает гораздо медленнее и подходит только для выборок небольшого размера. 

Количество проходов по данным для ```SGD``` задаётся с помощью параметра ```--passes```

**Параметры оптимизации**

Обновление весов происходит на каждом объекте:
<br/>

$$w_{t+1} = w_{t} + \eta_t \nabla_{w}\ell(w_{t}, x_{t})$$
$$\eta_t = \lambda d^k \left( \frac{t_0}{t_0 + t} \right)^p$$

где $t$ - номер объекта при обучении, $k$ - номер эпохи. Остальные параметры задаются следующим образом:

* $\lambda$: ```-l```
* $d$: ```--decay_learning_rate```
* $t_0$: ```--initial_t```
* $p$: ```--power_t```


**Функция потерь** задаётся через ```--loss_function```

**Регуляризация** задаётся через два флага ```--l1``` и ```--l2```

**Квадратичные признаки**

* ```-q ab``` — создаёт квадратичные признаки, перемножая все признаки из областей видимости, названия которых начинаются на букву a и на букву b
* ```--ignore a``` — игнорирует все признаки из области видимости, название которой начинается на букву a

**Подбор гиперпараметров**

* Инструмент для [подбора гиперпараметров](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Using-vw-hypersearch)

**Задание 3 (2 балла)**. Давайте попробуем потрогать руками сами! Для начала, преобразуйте данные, которые мы использовали в формат, который принимает VW.

In [110]:
def entry_to_vw(feature, feature_name, label=None):
    return str(label+1 or '') + ' |' + feature_name + ' ' + ' '.join(re.findall('\w{3,}', feature.lower())) + '\n'

In [111]:
def dataset_to_vw_file(X, y, filename='dataset'):
    amount = len(X)
    with open(f'{filename}.vw', 'w') as f:
        for i in range(amount):
            entry = entry_to_vw(X.iloc[i]['description'], 'description', y.iloc[i][0])
            f.write(entry)

In [112]:
X_text = pd.DataFrame({"description": description_lemma})
X_text.head()

,description
0,"aroma include tropical fruit , broom , brimsto..."
1,"this be ripe and fruity , a wine that be smoot..."
2,"tart and snappy , the flavor of lime flesh and..."
3,"pineapple rind , lemon pith and orange blossom..."
4,"much like the regular bottling from 2012 , thi..."


In [113]:
le = LabelEncoder()
le.fit(Y)
Y_numb = pd.DataFrame(le.transform(Y))

In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_text, Y_numb,  stratify=Y_numb, test_size=0.2, random_state=42)

In [139]:
dataset_to_vw_file(X_train, y_train, 'data.train')
dataset_to_vw_file(X_test, y_test, 'data.test')

У вас должно получиться [что-то в духе](https://yadi.sk/d/hUZ52TJxIKU-dw):

In [140]:
!head -n 1 data.train.vw

1 |description this newly release wine sultry and peppery with lip smack wave blackberry and raspberry bold and brood and ultimately delicious full bodied pron find balance between pron velvety texture and spike black pepper and leather


А теперь запустите VW из командной строки! Вам понадобятся следующие опции:
* ```-d``` - путь к данным
* ```-f``` - путь к месту, куда нужно сохранить модель
* ```--loss_function``` - функция потерь (давайте заиспользуем logistic)
* ```--oaa``` - для мультиклассовой классификации one-against-all количество классов
* ```--passes``` - количество итераций стохастического градиентного спуска (пусть будет 100)
* ```-b``` - размер хеш-функции, давайте сделаем 28
* добавим опции ```-c``` ```-k``` (для использования кэша)

In [141]:
!vw -d data.train.vw -f vw_model --loss_function logistic --oaa 3 --passes 100 -b 28 -c -k

final_regressor = vw_model
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = data.train.vw.cache
Reading datafile = data.train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1       36
0.500000 1.000000            2            2.0        3        1       46
0.750000 1.000000            4            4.0        3        1       39
0.625000 0.500000            8            8.0        2        3       21
0.812500 1.000000           16           16.0        1        2       42
0.531250 0.250000           32           32.0        3        3       28
0.468750 0.406250           64           64.0        3        3       41
0.421875 0.375000          128          128.0        1        1       33
0.410156 0.398438          256          256.0        3     

А теперь запустим модель на тесте:

* ```-d``` - путь к данным
* ```-i``` — путь до готовой модели
* ```-t``` — не обучаться, только вернуть предсказания
* ```-r``` — путь до файла для записи сырых предсказаний (```-p``` - выдает предсказанные классы)  
* ```--quite``` — не выводить никакую информацию в консоль

In [142]:
!vw -d data.test.vw -i vw_model -t -r predictions.txt -p predicted_labels.txt

only testing
predictions = predicted_labels.txt
raw predictions = predictions.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data.test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        2        3       33
0.500000 0.000000            2            2.0        3        3       30
0.750000 1.000000            4            4.0        2        3       31
0.375000 0.000000            8            8.0        3        3       34
0.312500 0.250000           16           16.0        1        3       59
0.218750 0.125000           32           32.0        3        3       34
0.187500 0.156250           64           64.0        3        3       43
0.226562 0.265625          128          128.0        1        1       28
0.234375 0.242188          256          256.0        3  

А теперь считаем предсказания и посчитаем качество!

In [143]:
with open('predicted_labels.txt') as pred_file:
    y_pred = [float(label) for label in pred_file.readlines()]

In [149]:
print ('Accuracy: ', accuracy_score(y_test + 1, y_pred))

Accuracy:  0.7506058857472591


# Полезные ссылки
* https://github.com/VowpalWabbit/vowpal_wabbit - официальный репозиторий VW
* https://www.kaggle.com/kashnitsky/vowpal-wabbit-tutorial-blazingly-fast-learning - отличный туториал
* https://github.com/VowpalWabbit/vowpal_wabbit/tree/master/python - питонячая обёртка над VW